# How to apply the Kelly Criterion for portfolio optimization in Python

The Kelly Criterion is a well-known betting strategy, which maximizes expected return with the constraint that asymptotic probability of ruin (loss of all wealth) is zero. In this article, we are going to apply this strategy to BIST 30 stocks with weekly re-balancing and measure the ~3 year out of sample performance against the official index.

We introduce a portfolio consisting of $k$ assets where allocation fraction to any asset $i$ is equal to $f_i$. Then, our portfolio return for a single time interval is:

$r_P = (r + f^T (\mu - \large\mathbb{1} \normalsize r))$

where,

$r_P:$ Portfolio expected return

$r:$ The risk-free rate, where we are going to use **TLREF** rates

$f:$ Allocation weights vector, of shape (k, 1) where k is the number of assets, to be optimized later

$\mu:$ Mean returns vector, of shape (k, 1), to be calculated from data

$\large\mathbb{1} \normalsize :$ A vector of 1's, of shape (k, 1)

Thus, the term $(\mu - \large\mathbb{1} \normalsize r)$ is the **excess returns** vector of $k$ assets and it is weighted by the allocations vector $f$ to result in the expected return of the portfolio. We would like to maximize the $r_P$ value. However, if we only considered this term for the optimization process we would have no control over the portfolio variance. Our goal is to maximize the expected **Sharpe Ratio** of our portfolio, which is equal to:

$\text{SR}_P = \Large\frac{r_P - r}{\sigma_P^2} \normalsize = \Large\frac{f^T (\mu - \large\mathbb{1} \normalsize r)}{\sqrt{f^T \Sigma f}} $

where,

$\text{SR}_P:$ Sharpe ratio of the portfolio

$\Sigma:$ Covariance matrix of the assets in the portfolio, of shape (k, k)

$\sigma_P^2 = f^T \Sigma f:$ Variance of the portfolio

Moreover, portfolio optimization problems are usually constrained with no short-selling and no leverage conditions, to get more applicable results. Thus, we re-formulate our optimization problem in the following way, which accounts for both the maximization of portfolio expected return and minimization of portfolio variance as well as the necessary constraints:

$max_f \quad (r + f^T (\mu - \large\mathbb{1} \normalsize r) - \large\frac{f^T \Sigma f}{2}) $

with constraints:

$\sum\limits_{i=1}^{N} f_i \le 1: \qquad$ allocation weights for $N$ assets should be less than 1, no leverage is allowed. $(1-\sum\limits_{i=1}^{N} f_i)$ fraction of allocations is invested into the risk-free rate $r$

$0 \le f_i \le 1: \qquad$ any allocation weight should be between 0 and 1, no short-selling is allowed.

After we designed our optimization objective and necessary constraints, we move on to the programming part:

First, we download 1000 calendar days of Close price data for the BIST30 stocks using the **yfinance** library.

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.optimize import minimize
import datetime as dt
import matplotlib.pyplot as plt

tickers = [
    "AEFES", "AKBNK", "ASELS", "ASTOR", "BIMAS", "CIMSA", "EKGYO", "ENKAI",
    "EREGL", "FROTO", "GARAN", "GUBRF", "ISCTR", "KCHOL", "KOZAL", "KRDMD",
    "MGROS", "PETKM", "PGSUS", "SAHOL", "SASA", "SISE", "TAVHL", "TCELL",
    "THYAO", "TOASO", "TTKOM", "TUPRS", "ULKER", "YKBNK"
]

BIST_suffix = ".IS"  # add the BIST suffix for yfinance

tickers = [ticker + BIST_suffix for ticker in tickers]

end_date = dt.date(2025, 8, 1)
days_delta = dt.timedelta(days=1000)
start_date = end_date - days_delta

price_data = yf.download(tickers=tickers, start=start_date, end=end_date, interval="1d")["Close"].dropna()
print()
print("Price Data shape:", price_data.shape)
(price_data)

C:\Users\Atakan\AppData\Local\Temp\ipykernel_10484\1873382085.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(tickers=tickers, start=start_date, end=end_date, interval="1d")["Close"].dropna()
[*********************100%***********************]  30 of 30 completed


Price Data shape: (629, 30)


Ticker,AEFES.IS,AKBNK.IS,ASELS.IS,ASTOR.IS,BIMAS.IS,CIMSA.IS,EKGYO.IS,ENKAI.IS,EREGL.IS,FROTO.IS,...,SASA.IS,SISE.IS,TAVHL.IS,TCELL.IS,THYAO.IS,TOASO.IS,TTKOM.IS,TUPRS.IS,ULKER.IS,YKBNK.IS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-18,3.440687,14.763787,31.414104,13.418216,127.910126,12.226055,7.928988,29.565042,19.856794,45.325340,...,6.601047,41.294952,88.699997,35.113743,147.154205,135.881912,23.400000,61.359631,40.776920,8.487298
2023-01-19,3.461630,14.959390,30.741478,14.755158,128.766647,11.981402,7.938405,28.765495,19.749144,44.875698,...,6.477613,42.227844,89.500000,34.778255,152.582428,139.396118,22.959999,61.160686,39.906963,8.522442
2023-01-20,3.467615,15.911891,31.115160,16.228722,128.766647,11.902056,8.126741,27.766062,20.062311,45.210728,...,6.466879,42.208809,89.900002,36.474304,150.904602,142.742950,23.360001,62.375320,39.333160,8.935385
2023-01-23,3.392817,15.792830,30.342888,17.848667,130.479752,11.749973,7.957238,26.984690,19.435976,45.422321,...,5.822875,40.666676,89.099998,36.884335,149.226791,139.479782,22.620001,63.820297,39.666336,8.996886
2023-01-24,3.356914,15.571712,31.389193,19.614992,130.289398,11.425974,7.702983,27.293602,19.132593,45.201912,...,5.779941,40.457245,87.349998,37.760319,147.746368,138.224701,23.180000,65.139633,38.018974,8.803595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-25,15.230000,68.000000,183.199997,102.599998,511.000000,50.400002,19.440001,69.150002,26.940001,92.400002,...,3.120000,37.299999,254.500000,92.500000,293.000000,219.399994,56.500000,167.100006,106.800003,33.299999
2025-07-28,15.190000,67.150002,181.300003,103.699997,515.000000,49.200001,19.379999,69.199997,26.719999,92.949997,...,3.130000,36.680000,254.000000,92.550003,290.250000,217.199997,55.400002,164.500000,106.300003,32.639999
2025-07-29,15.100000,67.150002,179.300003,103.099998,517.000000,48.799999,19.889999,66.550003,26.420000,91.500000,...,3.080000,36.400002,248.500000,91.099998,285.250000,214.000000,53.849998,162.600006,109.300003,32.860001


Then, we load the **TLREF** .xlsx file into pandas, and prepare it to calculate risk-free interval returns :

In [2]:
from pathlib import Path

def read_excel(filename):
    script_folder = Path().resolve()
    filepath = str(script_folder) + filename
    data = pd.read_excel(filepath, index_col="Date")
    return data

# Read excel file
tlref_data = read_excel(r"\TLREFORANI_D.xlsx")

# clean dates
dates = tlref_data.index
dates = [dt.date(*[int(value) for value in date.split("/")[::-1]]) if type(date) == str else dt.date(date.year, date.day, date.month) for date in dates]

# Calculate stock returns
returns_data = price_data.pct_change().shift(-1).dropna()

# Align the TLREF dataset with stocks dataset, calculate TLREF interval returns
tlref_data.index = pd.to_datetime(dates)
tlref_data = tlref_data.loc[returns_data.index, :]  # align date indexes with the stock returns dataset
daycounts = (tlref_data.index[1:] - tlref_data.index[:-1]).days.to_numpy()  # calculate calendar days between each business day
tlref_data = tlref_data.iloc[:-1, :]  # omit last day to align with daycounts shape

tlref_data.loc[:, "r"] = tlref_data.to_numpy().squeeze()/100 * daycounts/365
# Merge the dataframes
merged_df = pd.merge(returns_data, tlref_data.loc[:, "r"], how="inner", left_index=True, right_index=True)
merged_df

,AEFES.IS,AKBNK.IS,ASELS.IS,ASTOR.IS,BIMAS.IS,CIMSA.IS,EKGYO.IS,ENKAI.IS,EREGL.IS,FROTO.IS,...,SISE.IS,TAVHL.IS,TCELL.IS,THYAO.IS,TOASO.IS,TTKOM.IS,TUPRS.IS,ULKER.IS,YKBNK.IS,r
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-18,0.006087,0.013249,-0.021412,0.099636,0.006696,-0.020011,0.001188,-0.027044,-0.005421,-0.009920,...,0.022591,0.009019,-0.009554,0.036888,0.025862,-0.018803,-0.003242,-0.021335,0.004141,0.000247
2023-01-19,0.001729,0.063672,0.012156,0.099868,0.000000,-0.006622,0.023725,-0.034744,0.015857,0.007466,...,-0.000451,0.004469,0.048768,-0.010996,0.024010,0.017422,0.019860,-0.014379,0.048454,0.000249
2023-01-20,-0.021570,-0.007483,-0.024820,0.099820,0.013304,-0.012778,-0.020857,-0.028141,-0.031219,0.004680,...,-0.036536,-0.008899,0.011242,-0.011118,-0.022860,-0.031678,0.023166,0.008471,0.006883,0.000779
2023-01-23,-0.010582,-0.014001,0.034483,0.098961,-0.001459,-0.027574,-0.031953,0.011448,-0.015609,-0.004852,...,-0.005150,-0.019641,0.023749,-0.009921,-0.008998,0.024757,0.020673,-0.041530,-0.021484,0.000260
2023-01-24,-0.015151,-0.031131,-0.021429,0.099502,-0.003652,0.008680,0.006112,-0.027963,-0.010742,-0.012288,...,-0.012706,0.016600,-0.014314,0.014028,-0.028450,-0.023296,0.007877,-0.036027,-0.029940,0.000261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-23,-0.013933,0.009622,0.013881,0.029186,0.018528,0.013811,0.003041,-0.003647,0.002219,0.010177,...,0.005319,-0.027514,-0.001060,0.005093,0.044049,0.008795,0.009691,-0.003673,0.015207,0.001251
2025-07-24,-0.021837,-0.002933,0.003286,0.020896,0.004916,-0.004936,-0.017686,0.012445,-0.005904,-0.020148,...,-0.013228,-0.006829,-0.018568,-0.010135,-0.015260,-0.014821,0.002400,-0.015668,-0.002397,0.001252
2025-07-25,-0.002626,-0.012500,-0.010371,0.010721,0.007828,-0.023810,-0.003086,0.000723,-0.008166,0.005952,...,-0.016622,-0.001965,0.000541,-0.009386,-0.010027,-0.019469,-0.015560,-0.004682,-0.019820,0.003521


After our datasets are prepared, we can begin the calculations. We are going to use monthly intervals to calculate rolling returns and covariances. So, our first mean vector will be the average of the January 2023 of daily asset returns, then the average of February 2023, and so on. This also applies to the rolling covariance matrix as well. After we calculate our rolling monthly means and covariances, we'll get an optimized allocation vector $f^*$ for the beginning of the next month.

The mean of daily returns $\mu$ for each month:

In [3]:
mu_array_with_r = merged_df.resample("ME").mean()
print("The mean of daily returns for each month (including r):")
print("mu_array_with_r.shape:", mu_array_with_r.shape)
print(mu_array_with_r.iloc[-5:, -5:])  # show last 5x5 slice

The mean of daily returns for each month (including r):
mu_array_with_r.shape: (31, 31)
            TTKOM.IS  TUPRS.IS  ULKER.IS  YKBNK.IS         r
Date                                                        
2025-03-31  0.000494  0.005466  0.001688 -0.011733  0.001814
2025-04-30  0.005423 -0.003873 -0.002820 -0.001963  0.001948
2025-05-31  0.001720 -0.000177 -0.001232  0.005962  0.002075
2025-06-30  0.006051  0.007653  0.005159  0.011697  0.001990
2025-07-31 -0.005702  0.007874  0.001985  0.002596  0.001801


We also create an asset covariance matrix $\Sigma$ for each month and store them in a dictionary with the last date as its key:

In [4]:
# group by each year+month, then compute .cov() on the returns in that bucket
cov_array = {
    period: group.cov()
    for period, group in merged_df.iloc[:, :-1].groupby(pd.Grouper(freq="ME"))  # omit the risk-free rate from covariance calculations
}
last_date = mu_array_with_r.index[-1]
print("Last date:")
print(last_date)
print()
print("Covariance matrix for the last date:")
print(cov_array[last_date].round(4))  # 30x30 covariance matrix for the last date

Last date:
2025-07-31 00:00:00

Covariance matrix for the last date:
          AEFES.IS  AKBNK.IS  ASELS.IS  ASTOR.IS  BIMAS.IS  CIMSA.IS  \
AEFES.IS    0.0006    0.0003    0.0001    0.0003    0.0001    0.0002   
AKBNK.IS    0.0003    0.0003   -0.0000    0.0002    0.0001    0.0002   
ASELS.IS    0.0001   -0.0000    0.0004    0.0001   -0.0001    0.0000   
ASTOR.IS    0.0003    0.0002    0.0001    0.0004    0.0002    0.0001   
BIMAS.IS    0.0001    0.0001   -0.0001    0.0002    0.0002    0.0001   
CIMSA.IS    0.0002    0.0002    0.0000    0.0001    0.0001    0.0003   
EKGYO.IS    0.0002    0.0002    0.0001    0.0001    0.0001    0.0001   
ENKAI.IS    0.0003    0.0001    0.0002    0.0002    0.0001    0.0001   
EREGL.IS    0.0001    0.0002    0.0000    0.0001    0.0001    0.0001   
FROTO.IS    0.0001    0.0001   -0.0001    0.0001    0.0001    0.0001   
GARAN.IS    0.0003    0.0003   -0.0000    0.0002    0.0001    0.0002   
GUBRF.IS    0.0003    0.0002    0.0001    0.0002    0.0002    0.000

Finally, for each month-end data we have here, we are going to calculate the optimal allocation vector $f$. Then, we'll store them inside a dataframe. Let's remember our objective function to be maximized:

$(r + f^T (\mu - \large\mathbb{1} \normalsize r) - \large\frac{f^T \Sigma f}{2})$

Because we are going to use the SciPy minimize function, it is effectively the same problem to minimize the negative of the above function. So, our final objective function will be:

$g(f) = -(r + f^T (\mu - \large\mathbb{1} \normalsize r) - \large\frac{f^T \Sigma f}{2})$

where the optimization problem is to maximize function $g(.)$ with respect to the variable $f$ with constraints

* $\sum\limits_{i=1}^{N} f_i \le 1$

* $0 \le f_i \le 1$

In [ ]:
def calc_optimum_allocations(mu_array_with_r, cov_array):
  k = merged_df.iloc[:, :-1].shape[1]  # omit the risk-free rate, calculate asset count
  print("k =", k)

  # Assign initial value for f
  f_init = np.array([1/k for asset in range(k)])
  f_optimum_dict = {}
  
  for date in mu_array_with_r.index:
    r = mu_array_with_r.loc[date, "r"]  # date's mean risk-free return
    mu = (mu_array_with_r.loc[date, :]).iloc[:-1]  # date's mean daily returns vector, omitting the risk-free rate
    Sigma = cov_array[date]  # date's covariance matrix of daily returns


    # Define the objective function to be minimized
    def g(f):
      ones_vector = np.ones(k)
      col_f = f[:, np.newaxis]
      return -(r + np.dot(f, (mu - ones_vector*r)) - 0.5 * (col_f.T @ Sigma @ col_f))


    # Assign the constraint and bound
    cons = ({'type': 'ineq', 'fun': lambda f:  1 - f.sum()})
    bnds = tuple((0, 1) for f_i in f_init)

    # Optimize
    res = minimize(g, f_init, constraints=cons, bounds=bnds, tol=1e-12)
    f_optimum = np.round(res.x, 4)

    f_optimum_dict[date] = f_optimum

  f_optimum_df = (pd.DataFrame(f_optimum_dict)).transpose()
  return f_optimum_df

f_optimum_df = calc_optimum_allocations(mu_array_with_r=mu_array_with_r, cov_array=cov_array)
print(f_optimum_df.iloc[:5, :5])  # show the first 5x5 slice
print("f_optimum_df.shape:", f_optimum_df.shape)

k = 30
                0    1       2       3    4       5       6      7    8    9   \
2023-01-31  0.0000  0.0  0.0000  1.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-02-28  0.0000  0.0  0.0000  1.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-03-31  0.0000  0.0  0.0000  0.4460  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-04-30  0.0000  0.0  0.0000  0.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-05-31  0.0000  0.0  0.0000  0.0000  0.0  1.0000  0.0000  0.000  0.0  0.0   
2023-06-30  0.0000  0.0  0.0000  0.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-07-31  0.0000  0.0  0.0000  1.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-08-31  0.0000  0.0  0.0000  0.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-09-30  0.0000  0.0  0.0000  0.0000  0.0  0.0521  0.0000  0.000  0.0  0.0   
2023-10-31  0.0000  0.0  0.0000  0.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-11-30  0.0000  0.0  0.0000  0.0000  0.0  0.0000  0.0000  0.000  0.0  0.0   
2023-12-31  1.0000  0

After optimizing for the allocations, we check for the shape of the optimum $f$ matrix and we see that it makes sense. We have 31 months and 30 assets. So each row of the resulting matrix is the optimal asset allocation for a specific month. Now we move on to backtest our optimum allocations to see how it performs against the BIST30 benchmark:

In [6]:
def calc_daily_portfolio_returns(merged_df, f_optimum_matrix):
    ones_vector = np.ones(k)
    col_f = f[:, np.newaxis]
    return (r + np.dot(f, (mu - ones_vector*r)))